In [1]:
from string import printable, whitespace
from random import random, choice, choices, randint
from time import time

characters = ''.join(c for c in printable if c not in whitespace) + ' '

In [2]:
class DNA:
    def __init__(self, dna):
        self.dna = dna
        
    def mutate(self, mutation_rate=.01):
        self.dna = ''.join([gene if random() > mutation_rate else choice(characters) for gene in self.dna])
    
    def crossover(self, partner, target=None):
        
        if not target:
            index = randint(0,len(self.dna))
            child = self.dna[:index] + partner.dna[index:]
        else:
            child = ''
            index = 0
            for self_gene, partner_gene in zip(self.dna, partner.dna):
                child += self_gene if self_gene == target[index] else partner_gene
                index += 1
        
        return DNA(child)

    def calculate_fitness(self, target):
        self.fitness = 0
        for self_gene, target_gene in zip(self.dna, target):
            if self_gene == target_gene:
                self.fitness += 1
        self.fitness /= len(self.dna)
        self.fitness = self.fitness**4
    
    def __str__(self):
        return self.dna

In [3]:
class Population:
    def __init__(self, target, population_size=1000, mutation_rate=.01, crossover_bias=False):
        self.target = target
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_bias = crossover_bias
        self.generation = 0
        self.population = []
        self.start_time = time()
        self.generate()
    
    def generate(self):
        
        first_generation = len(self.population) != self.population_size
        self.new_population = []
        self.member_dna = []
        self.best_fitness = 0
        self.generation += 1

        for _ in range(self.population_size):
            if first_generation:
                child = DNA(''.join(choice(characters) for _ in range(len(self.target))))
                self.population.append(child)
            else:
                parent_a, parent_b = self.selection()
                child = parent_a.crossover(parent_b, target=self.target if self.crossover_bias else None)
                child.mutate(self.mutation_rate)

                self.new_population.append(child)
            
            child.calculate_fitness(target)
            if child.fitness > self.best_fitness:
                self.best_fitness = child.fitness
                self.best_member = child
                
            self.member_dna.append(child.dna)
        
        self.population = self.new_population if self.new_population else self.population
        
        self.fitnesses = [member.fitness for member in self.population]
        self.average_fitness = sum(self.fitnesses) / len(self.population)

    def selection(self):
        
        # Select two parents from the population based on their fitness
        parent_a, parent_b = choices(self.population, weights=self.fitnesses, k=2)
        
        return parent_a, parent_b
    
    def __str__(self):
        self.end_time = int(time() - self.start_time)
        return (f"Generation: {self.generation}\t Average Fitness: {self.average_fitness:.0%}\t " +
                f"Best: {self.best_member.dna}\t Time: {self.end_time}s")

In [4]:
target = "To be or not to be, that is the question."
pool = Population(target, crossover_bias=True)


while target not in pool.member_dna:
    print(pool)
    pool.generate()
else:
    print('-' * 100)
    print(pool)


Generation: 1	 Average Fitness: 0%	 Best: ?6>kf->g?yP[&wo ARK(11!P qZu:SQCJI)+'.ldA	 Time: 0s
Generation: 2	 Average Fitness: 0%	 Best: T:mcM oqrsoY2^[%Q8[bf[SEBl{1l vl*(.rtQonp	 Time: 0s
Generation: 3	 Average Fitness: 0%	 Best: T{ ~D oHx}osr7oIUm} :ZMQIe-\a6_ P!+stCon%	 Time: 0s
Generation: 4	 Average Fitness: 2%	 Best: Tr)4: o%r}oN to 'l` tMn* _7@t'M p9eEtion.	 Time: 0s
Generation: 5	 Average Fitness: 7%	 Best: T<LUe or eo! +o ;e, ttaC Xs the qFQ1tion.	 Time: 0s
Generation: 6	 Average Fitness: 22%	 Best: To be or not to 6ez thaQ Ss 0he -Aest[on.	 Time: 1s
Generation: 7	 Average Fitness: 45%	 Best: To be or not to be, that Ps the quJstion.	 Time: 1s
Generation: 8	 Average Fitness: 65%	 Best: To be or not to be, that As the question.	 Time: 1s
Generation: 9	 Average Fitness: 77%	 Best: To be or not to ;e, that is the question.	 Time: 1s
----------------------------------------------------------------------------------------------------
Generation: 10	 Average Fitness: 82%	 Best: To be